# DSO 560 Project

Yian L.

## Import packages and dataset

In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from random import randint
import scipy
from numpy import array, argmax, asarray, zeros
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking

In [2]:
brands = pd.read_csv('behold_brands USC.csv')

In [3]:
brands.head()

,brand_id,brand,brand_value,bio,quote,quote_attribute,intro,lifestyle_copy,short_bio,listing_bio
0,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Handmade / Artisan Crafted,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
1,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Sustainable,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
2,01ESKR0CH2KYC7KBNTN0S38EQA,Mari Giudicelli,Women Owned,NaN,NaN,NaN,NaN,NaN,NaN,Behold Mari Giudicelli! This Brazilian shoe de...
3,01ESVD80YYAJQEQPADZ7HTHZXA,Warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Behold Warm! Founded by specialty store owner,..."
4,01EFJFZ329YVC8SK87R2BNJGV0,Vaara,Emerging,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
products = pd.read_excel('Behold+product+data+04262021.xlsx')

In [5]:
products.head()

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,NaN,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,NaN,NaN,CATIBA PRO Skate Black Suede and Canvas Contra...,False


In [6]:
outfit = pd.read_csv('outfit_combinations USC.csv')

In [7]:
outfit.head()

,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,Tory Burch,Penelope Mid Cap Toe Pump
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt


In [8]:
tags = pd.read_csv('usc_additional_tags USC.csv')

In [9]:
tags.head()

,product_id,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic


## Fill in missing values

I will fill in missing values using "Unknown"

In [10]:
products_data = products[['product_id', 'brand', 'brand_category',
                          'name', 'details', 'description', 'brand_description', 'brand_name']]

In [11]:
products_data.isna().sum()

product_id               0
brand                    0
brand_category         459
name                     1
details              52155
description          10117
brand_description    10121
brand_name               1
dtype: int64

In [12]:
products_data = products_data.fillna("Unknown")

In [13]:
products_data.dtypes

product_id           object
brand                object
brand_category       object
name                 object
details              object
description          object
brand_description    object
brand_name           object
dtype: object

In [14]:
products_data.shape

(61355, 8)

In [15]:
brands.shape

(162, 10)

In [16]:
brands.isnull().sum()

brand_id           0
brand              0
brand_value        8
bio                5
quote              5
quote_attribute    8
intro              8
lifestyle_copy     6
short_bio          5
listing_bio        5
dtype: int64

In [17]:
brands = brands.fillna("Unknown")

In [18]:
outfit.shape

(5291, 5)

In [19]:
outfit.isnull().sum()

outfit_id            0
product_id           0
outfit_item_type     0
brand                0
product_full_name    0
dtype: int64

In [20]:
tags.shape

(97420, 4)

In [21]:
tags.isnull().sum()

product_id          0
product_color_id    0
attribute_name      0
attribute_value     0
dtype: int64

## Preprocess the text

For product data

In [22]:
products_data_2 = products_data.copy()

In [23]:
products_data['description'][0]

'Our signature khadi shirt\navailable in black and white\nEasy to wear from beach to city. We promise this top will be your go-to warm\nweather item. Perfect under a blazer. Hand loomed woven stripe in khadi cotton.\n Slightly sheer and gets softer with every wash. Ships First week of April\nCOLOR: WHITE OR BLACK\n—  Length 27"  and  Width  26.5" —  One size fits most —  Grid khadi cotton'

### Remove punctuations

In [24]:
col_list = ['brand', 'brand_category', 'name', 'details', 'description', 'brand_description', 'brand_name']
# punc = "!\"'#$%&\'()*+,./:;<=>?[\\]^_`{|}~’”•`'''``"

for col in col_list:
    products_data_2[col] = products_data_2[col].str.strip()
    products_data_2[col] = products_data_2[col].str.lower()
    
    products_data_2[col] = products_data_2[col].str.replace('\n', ' ')
    
    products_data_2[col] = products_data_2[col].str.replace(r"\b\w+n't\b", 'not')
    # changed all negative meaning words (doesn't, isn't, hadn't, etc.) to "not"
    
    products_data_2[col] = products_data_2[col].str.replace(r'[^\w\s]', ' ')
    # removed punctuations

In [25]:
products_data_2['description'][0]

'our signature khadi shirt available in black and white easy to wear from beach to city  we promise this top will be your go to warm weather item  perfect under a blazer  hand loomed woven stripe in khadi cotton   slightly sheer and gets softer with every wash  ships first week of april color  white or black    length 27   and  width  26 5     one size fits most    grid khadi cotton'

### Remove stopwords and numbers

In [26]:
products_data_3 = products_data_2.copy()

In [27]:
stopwords = set(stopwords.words("English"))

In [28]:
def remove_stopwords(text):
    words = word_tokenize(str(text))
    words_no_stopwords = [word for word in words if word not in stopwords and not word.isdigit()]
    words_no_stopwords_str = " ".join(words_no_stopwords)
    return words_no_stopwords_str

In [29]:
for col in col_list:
    products_data_3[col] = products_data_3[col].apply(remove_stopwords)

In [30]:
products_data_3['description'][0]

'signature khadi shirt available black white easy wear beach city promise top go warm weather item perfect blazer hand loomed woven stripe khadi cotton slightly sheer gets softer every wash ships first week april color white black length width one size fits grid khadi cotton'

### Lemmatization

In [31]:
products_data_4 = products_data_3.copy()

In [32]:
# https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return ' '.join(lemmatized_sentence)

In [33]:
for col in col_list:
    products_data_4[col] = products_data_4[col].apply(lambda x: lemmatize_sentence(x))

KeyboardInterrupt: 

In [ ]:
products_data_4['description'][0]

In [ ]:
products_data_4.head()

## Feature engineering

I don't use extra data files but use `product_data` only in this project.

## Embeddings and Model

In [ ]:
products_data_5 = products_data_4.copy()

Combine the colomn details and description together.

In [ ]:
products_data_5['details+description'] = products_data_5['details'] + " " + products_data_5['description']

In [ ]:
products_data_5.head()

In [ ]:
products_data_6 = products_data_5[['brand', 'brand_category', 'name', 'details', 
                                   'description', 'details+description']]

In [ ]:
products_data_6.head()

In [ ]:
details_description_doc = list(products_data_6['details+description'])

In [ ]:
# so this means that all tokens that are not found in the vocabulary are going to be marked as UNKNOWN_TOKEN
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(details_description_doc)

In [ ]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

# integer encode the documents
encoded_docs = integer_encode_documents(details_description_doc, tokenizer)

In [ ]:
MAX_SEQUENCE_LENGTH = 64

In [ ]:
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
padded_docs.shape

Find out the top 30 bands and set other brands as 'others'

In [ ]:
products_data_6['brand'].value_counts()[:30]

In [ ]:
label_lst = products_data_6['brand'].value_counts().index
label_lst = label_lst[:30]
label_lst

In [ ]:
labels = [label if label in label_lst else 'others' for label in products_data_6['brand']]

In [ ]:
encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2)

In [ ]:
X_test.shape

In [ ]:
VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

In [ ]:
def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [ ]:
def make_lstm_classification_model(plot=False):
    model =  keras.models.Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(units=32, input_shape=(1, MAX_SEQUENCE_LENGTH)))
    model.add(Dense(16))
    model.add(Dense(31, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    
    if plot:
        plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
model = make_lstm_classification_model()

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.1,
                    epochs=20, verbose=1)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

## Recommendation

### Check Out Different Types of Categories

First, we checked out the file `outfit_combinations USC.csv` to see the categories.

In [34]:
outfit['outfit_item_type'].unique()

array(['bottom', 'top', 'accessory1', 'shoe', 'onepiece', 'accessory2',
       'accessory3'], dtype=object)

There are three kinds of accessory 'accessory1', 'accessory2', 'accessory3'.

The reason for three kinds of accessory was that for each outfit_id, there might be more than one accessories.

We renamed three kinds of accessory to 'accessory'. After this step, there should be 5 item types: bottom, top, accessory, shoe, onepiece.

We also found out some product in the outfit dataframe may have the wrong category. 

For example, the item with product_id 01DT0C8NM9KG2EF0A286VZRETE has the name as 'Tank top in Re-Imagined Silk'. The name seemed to indicate that this product belonged to 'top' but the given category was 'accessory1'.

In [35]:
outfit[outfit['outfit_item_type'] == 'accessory1'].head()

,outfit_id,product_id,outfit_item_type,brand,product_full_name
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel
7,01DMHCX50CFX5YNG99F3Y65GQW,01DMHCNT41E14QWP503V7CT9G6,accessory1,Nina,Crystal Clutch
10,01DMHRX35M2DPVYVQ1PNER4S4B,01DMHCNT41E14QWP503V7CT9G6,accessory1,Nina,Crystal Clutch
12,01DMHRYZDTF2NBJ13ZEARKHA3T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel
18,01DQ86EH3GMXAVKNECH2Z6FCSV,01DPGWYTS97KK213K1QNJV6610,accessory1,J.Crew,Bembien® Jeanne leather woven market tote bag


In [36]:
outfit[outfit['outfit_item_type'] == 'accessory2'].head()

,outfit_id,product_id,outfit_item_type,brand,product_full_name
21,01DQ8KWVX1GBJTPTVDAC6NQ9B4,01DPGZWS42TP4NJ1GGF50Z9DZF,accessory2,Veronica Beard,Matteo Dickey Jacket
30,01DQ8MQAVBFSGHJXCF5JCYJ7A6,01DPKN22N1X0RRJQYAYVTRPD75,accessory2,Reformation,Morgan Sweater
33,01DQ8NM7YPX63A6V7S0WC8N4X6,01DPH1A23QXSRMGAR9H3FRND2F,accessory2,Sole Society,Ragna Clutch
41,01DQ8XK5WH02037JGY1NA5W5JS,01DPGVQYVFSBEP5RK51CZTK5YB,accessory2,Frame,Velvet Stand Soutache Jacket
43,01DQBAVTX4DV62X94VAKV5GCC9,01DPGV73E6KQ1C31G7F1TGAWGB,accessory2,Frame,Patch Pocket Trench


In [37]:
outfit[outfit['outfit_item_type'] == 'accessory3'].head()

,outfit_id,product_id,outfit_item_type,brand,product_full_name
4720,01E4CKE4MA4TPT3GS4J1E1QTXY,01DS44MT6XDH3DDGJ80TQ6AJ2G,accessory3,Aleksandre Akhalkatsishvili,Asymmetric Cotton Trench Coat


We looked at the accessory category. There was only one product with `outfit_item_type` as 'accessory3'.

#### Renamed Three Kinds of Accessory to 'Accessory'

In [38]:
outfit_2 = outfit.copy()

In [39]:
col_lst = ['accessory1', 'accessory2', 'accessory3']

for i in range(len(outfit_2)):
    if outfit_2['outfit_item_type'][i] in col_lst:
        outfit_2.loc[i, 'outfit_item_type'] = 'accessory'

In [40]:
outfit_2['outfit_item_type'].unique()

array(['bottom', 'top', 'accessory', 'shoe', 'onepiece'], dtype=object)

### Do Data Cleaning and Feature Engineering

#### Fill in Missing Values 

We need to predict brand_category for those products without category before doing recommendation.

First, we filled in missing values in `Behold+product+data+04262021.xlsx` using 'Unknown'.

In [41]:
products_1 = products.copy()

In [42]:
products_1 = products_1.fillna("Unknown")

#### Select Features Manually and Do Feature Engineering

We selected two columns 'brand_category' and 'name' to predict the category of each product.

We also combined product information by adding 'brand', 'brand_category', 'name', 'details' and 'description' together for prediction and recommendation.

In [43]:
products_1['product_information'] = products_1['brand_category'].astype(str) + ' ' + \
                                    products_1['brand'].astype(str) + ' ' + \
                                    products_1['name'].astype(str) + ' ' + \
                                    products_1['details'].astype(str) + ' ' + \
                                    products_1['description'].astype(str)

In [44]:
products_2 = products_1.copy()

In [45]:
products_2 = products_2[['product_id', 'brand_category', 'name', 'product_information']]

In [46]:
products_2.head()

,product_id,brand_category,name,product_information
0,01EX0PN4J9WRNZH5F93YEX6QAF,Unknown,Khadi Stripe Shirt-our signature shirt,Unknown Two Khadi Stripe Shirt-our signature s...
1,01F0C4SKZV6YXS3265JMC39NXW,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,Unknown Collina Strada RUFFLE MARKET DRESS LOO...
2,01EY4Y1BW8VZW51BWG5VZY82XW,Unknown,IBI Slip On Raw Red Knit Sneaker Women,Unknown Cariuma IBI Slip On Raw Red Knit Sneak...
3,01EY50E27A0P5V6KCW01XPDB43,Unknown,IBI Slip On Black Knit Sneaker Women,Unknown Cariuma IBI Slip On Black Knit Sneaker...
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,Unknown Cariuma CATIBA PRO Skate Black Suede a...


#### Preprocess Text for Features

In [47]:
for col in ['brand_category', 'name', 'product_information']:
    products_2[col] = products_2[col].str.lower()
    products_2[col] = products_2[col].str.replace('\n', ' ')
    products_2[col] = products_2[col].str.replace(r"\b\w+n't\b", 'not')
    # changed all negative meaning words (doesn't, isn't, hadn't, etc.) to "not"
    products_2[col] = products_2[col].str.replace(r'[^\w\s]', ' ')
    # removed punctuations

In [48]:
products_3 = products_2.copy()

In [49]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words("English"))


def remove_stopwords(text):
    words = word_tokenize(str(text))
    words_no_stopwords = [word for word in words if word not in stopwords and not word.isdigit()]
    words_no_stopwords_str = " ".join(words_no_stopwords)
    return words_no_stopwords_str


for col in ['brand_category', 'name', 'product_information']:
    products_3[col] = products_3[col].apply(remove_stopwords)

In [50]:
products_4 = products_3.copy()

In [51]:
# https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return ' '.join(lemmatized_sentence)

In [52]:
for col in ['brand_category', 'name', 'product_information']:
    products_4[col] = products_4[col].apply(lambda x: lemmatize_sentence(x))

In [53]:
products_4.head()

,product_id,brand_category,name,product_information
0,01EX0PN4J9WRNZH5F93YEX6QAF,unknown,khadi stripe shirt signature shirt,unknown two khadi stripe shirt signature shirt...
1,01F0C4SKZV6YXS3265JMC39NXW,unknown,ruffle market dress loopy pink sistine tomato,unknown collina strada ruffle market dress loo...
2,01EY4Y1BW8VZW51BWG5VZY82XW,unknown,ibi slip raw red knit sneaker woman,unknown cariuma ibi slip raw red knit sneaker ...
3,01EY50E27A0P5V6KCW01XPDB43,unknown,ibi slip black knit sneaker woman,unknown cariuma ibi slip black knit sneaker wo...
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,unknown,catiba pro skate black suede canvas contrast t...,unknown cariuma catiba pro skate black suede c...


In [54]:
products_5 = products_4.copy()

In [55]:
for col in ['brand_category', 'name', 'product_information']:
    products_5[col] = products_5[col].str.split(r'[\s\:\/]+')

Create a new column named 'outfit_type' and set the default as 'Unknown'

In [56]:
products_5['outfit_type'] = 'Unknown'

In [57]:
products_5.head()

,product_id,brand_category,name,product_information,outfit_type
0,01EX0PN4J9WRNZH5F93YEX6QAF,[unknown],"[khadi, stripe, shirt, signature, shirt]","[unknown, two, khadi, stripe, shirt, signature...",Unknown
1,01F0C4SKZV6YXS3265JMC39NXW,[unknown],"[ruffle, market, dress, loopy, pink, sistine, ...","[unknown, collina, strada, ruffle, market, dre...",Unknown
2,01EY4Y1BW8VZW51BWG5VZY82XW,[unknown],"[ibi, slip, raw, red, knit, sneaker, woman]","[unknown, cariuma, ibi, slip, raw, red, knit, ...",Unknown
3,01EY50E27A0P5V6KCW01XPDB43,[unknown],"[ibi, slip, black, knit, sneaker, woman]","[unknown, cariuma, ibi, slip, black, knit, sne...",Unknown
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,[unknown],"[catiba, pro, skate, black, suede, canvas, con...","[unknown, cariuma, catiba, pro, skate, black, ...",Unknown


### Predict Outfit Category for Products Without Category

#### Collect Keywords for Each Category Using Domain Knowledge

We collected a bunch of keywords for each category using our business domain knowledge. In this dataset, we found out some items did not belong to outfit, and we collected some words and set the category as 'others'.

First, we used brand_category of each product to match the keywords in each category. Second, we used name of the product to match. Last, we used product information of the product to match. If one product do not match any category, we would leave it as 'Unknown'.

To be specific, in every match, we created a dictionary for each records and counted the frequency of the keywords in each category, and set the most frequent category as the category of the product. 

If two or more categories had the same frequency, we would randomly pick one as the category of the product.

In [58]:
bottom_lst = ['pant', 'skirt', 'jean', 'trouser', 'short', 'tight', 'legging', 'culotte',
              'bootcut', 'leg', 'palazzo', 'bottom', 'ankle', 'capri', 'waistband',
              'waist', 'highwaist', 'beltless', 'straight']

one_piece_lst = ['dress', 'piece', 'jumpsuit', 'piece', 'bodysuit', 'slipdress', 'shirtdress',
                 'onepiece', 'onesie', 'gown', 'robe', 'romper', 'shortall', 'minidress',
                 'caftan', 'tunic', 'georgette', 'coverall', 'bikini', 'kimono', 'sundress']

shoe_lst = ['shoe', 'boot', 'sneaker', 'heel', 'foot', 'sandal', 'slipper', 
            'flip', 'flop', 'bootie', 'toe', 'pump', 'trainer', 'platform', 'oxford', 
            'mule', 'brogue', 'loafer', 'moccasin', 'flat', 'derby', 'slingback', 'clog',
            'heighten', 'shoetie']

top_lst = ['tank', 'top', 'blouse', 'shirt', 'tee', 'vest', 'blazer', 'crop', 
           'hoodie', 'hood', 'sweat', 'turtleneck', 'cardigan', 'camisole', 'sweatshirt', 
           'neck', 'sleeve', 'cami', 'boatneck', 'vneck', 'jersey', 'sleeves', 'tanktop',
           'sweatpants', 'bustier', 'sleeveless', 'neckline', 'crewneck', 'longsleeve']

accessory_lst = ['satchel', 'clutch', 'bag', 'tote', 'jacket', 'coat', 'scarf', 
                 'bra', 'bralette', 'belt', 'backpack', 'briefcase', 'purse', 'panty', 
                 'thong', 'belt', 'hat', 'bralett', 'hobo', 'eye', 'sunglasses', 'bib', 
                 'accessory', 'sunglass', 'lens', 'trench', 'wallet', 'earring', 'barrette',
                 'pullover', 'photo', 'card', 'band', 'felt', 'hand', 'necklace',
                 'shearling', 'cream', 'lip', 'balm', 'parka', 'mask', 'bracelet',
                 'sock', 'glasswear', 'cape', 'suit', 'bandana', 'lenses', 'lingerie',
                 'collar', 'apron', 'tie', 'strap', 'ring', 'napkin', 'shawl', 'sweater', 
                 'beret', 'sapphire', 'crossbody', 'neckband', 'headband', 'headgear',
                 'outerwear', 'wrist', 'cap', 'shirtjacket', 'windbreaker', 'glove', 'mitt',
                 'bangle', 'obi', 'stud', 'earing', 'overcoat', 'trenchcoat', 'watch', 'anklet',
                 'mitts', 'choker', 'pin', 'gloves']

others_lst = ['towel', 'vase', 'chair', 'candle', 'photo', 'card', 'book', 'lamp', 
              'pottery', 'plate', 'salt', 'pillow', 'table', 'bench', 'bed', 'table',
              'couch', 'baby', 'basket', 'crochet', 'coverlet', 'upholster', 'cushion',
              'makeup', 'ceramic', 'soap', 'antique', 'sofa', 'footbed', 'goblet', 'skateboard',
              'quilt', 'washcloth', 'comb', 'fragrance', 'mat', 'swimwear', 'swimsuit', 'wetsuit',
              'pillowcase', 'perfume', 'enamel', 'insole', 'shower', 'furniture', 'toiletry',
              'pilowcases']

#### Predict Outfit Category

In [59]:
for i in range(len(products_5)):
    outfit_dict = {}
    for j in products_5['brand_category'][i]:
        if j in bottom_lst:
            if 'bottom' not in outfit_dict.keys():
                outfit_dict['bottom'] = 1
            else:
                outfit_dict['bottom'] += 1
        elif j in one_piece_lst:
            if 'onepiece' not in outfit_dict.keys():
                outfit_dict['onepiece'] = 1
            else:
                outfit_dict['onepiece'] += 1
        elif j in shoe_lst:
            if 'shoe' not in outfit_dict.keys():
                outfit_dict['shoe'] = 1
            else:
                outfit_dict['shoe'] += 1
        elif j in top_lst:
            if 'top' not in outfit_dict.keys():
                outfit_dict['top'] = 1
            else:
                outfit_dict['top'] += 1
        elif j in accessory_lst:
            if 'accessory' not in outfit_dict.keys():
                outfit_dict['accessory'] = 1
            else:
                outfit_dict['accessory'] += 1
        elif j in others_lst:
            if 'others' not in outfit_dict.keys():
                outfit_dict['others'] = 1
            else:
                outfit_dict['others'] += 1
    if outfit_dict:
        # remove 'others' category if the keywords in a certain record match the 5 categories
        if len(outfit_dict) > 1 and 'others' in outfit_dict:
            del outfit_dict['others']
        products_5.loc[i, 'outfit_type'] = max(outfit_dict.items(), key=lambda x:x[1])[0]

In [60]:
len(products_5)

61355

In [61]:
len(products_5[products_5['outfit_type'] != 'Unknown'])

5540

The outfit types of 5540 records were predicted using the 'brand_category', which was around 9.0%.

In [62]:
for i in range(len(products_5)):
    if products_5.loc[i, 'outfit_type'] == 'Unknown':
        outfit_dict = {}
        for j in products_5['name'][i]:
            if j in bottom_lst:
                if 'bottom' not in outfit_dict.keys():
                    outfit_dict['bottom'] = 1
                else:
                    outfit_dict['bottom'] += 1
            elif j in one_piece_lst:
                if 'onepiece' not in outfit_dict.keys():
                    outfit_dict['onepiece'] = 1
                else:
                    outfit_dict['onepiece'] += 1
            elif j in shoe_lst:
                if 'shoe' not in outfit_dict.keys():
                    outfit_dict['shoe'] = 1
                else:
                    outfit_dict['shoe'] += 1
            elif j in top_lst:
                if 'top' not in outfit_dict.keys():
                    outfit_dict['top'] = 1
                else:
                    outfit_dict['top'] += 1
            elif j in accessory_lst:
                if 'accessory' not in outfit_dict.keys():
                    outfit_dict['accessory'] = 1
                else:
                    outfit_dict['accessory'] += 1
            elif j in others_lst:
                if 'others' not in outfit_dict.keys():
                    outfit_dict['others'] = 1
                else:
                    outfit_dict['others'] += 1
        if outfit_dict:
            # remove 'others' category if the keywords in a certain record match the 5 categories
            if len(outfit_dict) > 1 and 'others' in outfit_dict:
                del outfit_dict['others']
            products_5.loc[i, 'outfit_type'] = max(outfit_dict.items(), key=lambda x:x[1])[0]

In [63]:
len(products_5[products_5['outfit_type'] != 'Unknown'])

45923

The outfit types of 40383 records were predicted using the 'name', which was around 65.8%.

In [64]:
for i in range(len(products_5)):
    if products_5.loc[i, 'outfit_type'] == 'Unknown':
        outfit_dict = {}
        for j in products_5['product_information'][i]:
            if j in bottom_lst:
                if 'bottom' not in outfit_dict.keys():
                    outfit_dict['bottom'] = 1
                else:
                    outfit_dict['bottom'] += 1
            elif j in one_piece_lst:
                if 'onepiece' not in outfit_dict.keys():
                    outfit_dict['onepiece'] = 1
                else:
                    outfit_dict['onepiece'] += 1
            elif j in shoe_lst:
                if 'shoe' not in outfit_dict.keys():
                    outfit_dict['shoe'] = 1
                else:
                    outfit_dict['shoe'] += 1
            elif j in top_lst:
                if 'top' not in outfit_dict.keys():
                    outfit_dict['top'] = 1
                else:
                    outfit_dict['top'] += 1
            elif j in accessory_lst:
                if 'accessory' not in outfit_dict.keys():
                    outfit_dict['accessory'] = 1
                else:
                    outfit_dict['accessory'] += 1
            elif j in others_lst:
                if 'others' not in outfit_dict.keys():
                    outfit_dict['others'] = 1
                else:
                    outfit_dict['others'] += 1
        if outfit_dict:
            # remove 'others' category if the keywords in a certain record match the 5 categories
            if len(outfit_dict) > 1 and 'others' in outfit_dict:
                del outfit_dict['others']
            products_5.loc[i, 'outfit_type'] = max(outfit_dict.items(), key=lambda x:x[1])[0]

In [65]:
len(products_5[products_5['outfit_type'] == 'Unknown'])

5427

There were X records left as 'Unknown'. We would get rid of these records later.

In [66]:
products_6 = products_5.copy()

### Merge and Preprocess the Products Dataframe and Outfit Dataframe

We combined brand and name for each product in the outfit dataframe, and merged the products dataframe and outfit dataframe.

#### Do Data Cleaning and Feature Engineering for for Merged Dataframe

In [67]:
outfit_2['brand+name'] = outfit_2['brand'] + outfit_2['product_full_name']

In [68]:
products_with_outfit = pd.merge(products_6, outfit_2, on='product_id', how='outer')

Most products in the products dataframe did not have outfit information, but there were still some products in products dataframe with outfit information. 

There were also some products in the outfit dataframe without product information in the products dataframe. This is to say, these products appeared in the outfit dataframe but not in the products dataframe.

In [69]:
a = len(products) - products_with_outfit.outfit_id.isnull().sum()
b = len(outfit) - (len(products) - products_with_outfit.outfit_id.isnull().sum())
print(f'There were {a} existing products in the products dataframe with outfit information.')
print(f'There were {b} products with outfit in the outfit dataframe without product information.')

There were 769 existing products in the products dataframe with outfit information.
There were 4522 products with outfit in the outfit dataframe without product information.


In [70]:
products_with_outfit.head()

,product_id,brand_category,name,product_information,outfit_type,outfit_id,outfit_item_type,brand,product_full_name,brand+name
0,01EX0PN4J9WRNZH5F93YEX6QAF,[unknown],"[khadi, stripe, shirt, signature, shirt]","[unknown, two, khadi, stripe, shirt, signature...",top,NaN,NaN,NaN,NaN,NaN
1,01F0C4SKZV6YXS3265JMC39NXW,[unknown],"[ruffle, market, dress, loopy, pink, sistine, ...","[unknown, collina, strada, ruffle, market, dre...",onepiece,NaN,NaN,NaN,NaN,NaN
2,01EY4Y1BW8VZW51BWG5VZY82XW,[unknown],"[ibi, slip, raw, red, knit, sneaker, woman]","[unknown, cariuma, ibi, slip, raw, red, knit, ...",shoe,NaN,NaN,NaN,NaN,NaN
3,01EY50E27A0P5V6KCW01XPDB43,[unknown],"[ibi, slip, black, knit, sneaker, woman]","[unknown, cariuma, ibi, slip, black, knit, sne...",shoe,NaN,NaN,NaN,NaN,NaN
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,[unknown],"[catiba, pro, skate, black, suede, canvas, con...","[unknown, cariuma, catiba, pro, skate, black, ...",shoe,NaN,NaN,NaN,NaN,NaN


In [71]:
products_with_outfit_2 = products_with_outfit.copy()

In [72]:
products_with_outfit_2 = products_with_outfit_2[['product_id', 'brand_category', 'name',
                                                 'product_information', 'outfit_type', 'outfit_id', 
                                                 'outfit_item_type', 'brand+name']]

In [73]:
products_with_outfit_2 = products_with_outfit_2.fillna("Unknown")

In [74]:
for col in ['brand+name']:
    products_with_outfit_2[col] = products_with_outfit_2[col].str.lower()
    products_with_outfit_2[col] = products_with_outfit_2[col].str.replace('\n', ' ')
    products_with_outfit_2[col] = products_with_outfit_2[col].str.replace(r"\b\w+n't\b", 'not')
    # changed all negative meaning words (doesn't, isn't, hadn't, etc.) to "not"
    products_with_outfit_2[col] = products_with_outfit_2[col].str.replace(r'[^\w\s]', ' ')
    # removed punctuations

In [75]:
products_with_outfit_2['brand+name'] = products_with_outfit_2['brand+name'].apply(remove_stopwords)

products_with_outfit_2['brand+name'] = \
            products_with_outfit_2['brand+name'].apply(lambda x: lemmatize_sentence(x))

products_with_outfit_2['brand+name'] = products_with_outfit_2['brand+name'].str.split(r'[\s\:\/]+')

In [76]:
products_with_outfit_2.head()

,product_id,brand_category,name,product_information,outfit_type,outfit_id,outfit_item_type,brand+name
0,01EX0PN4J9WRNZH5F93YEX6QAF,[unknown],"[khadi, stripe, shirt, signature, shirt]","[unknown, two, khadi, stripe, shirt, signature...",top,Unknown,Unknown,[unknown]
1,01F0C4SKZV6YXS3265JMC39NXW,[unknown],"[ruffle, market, dress, loopy, pink, sistine, ...","[unknown, collina, strada, ruffle, market, dre...",onepiece,Unknown,Unknown,[unknown]
2,01EY4Y1BW8VZW51BWG5VZY82XW,[unknown],"[ibi, slip, raw, red, knit, sneaker, woman]","[unknown, cariuma, ibi, slip, raw, red, knit, ...",shoe,Unknown,Unknown,[unknown]
3,01EY50E27A0P5V6KCW01XPDB43,[unknown],"[ibi, slip, black, knit, sneaker, woman]","[unknown, cariuma, ibi, slip, black, knit, sne...",shoe,Unknown,Unknown,[unknown]
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,[unknown],"[catiba, pro, skate, black, suede, canvas, con...","[unknown, cariuma, catiba, pro, skate, black, ...",shoe,Unknown,Unknown,[unknown]


In [77]:
products_with_outfit_2.tail()

,product_id,brand_category,name,product_information,outfit_type,outfit_id,outfit_item_type,brand+name
65872,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,Unknown,Unknown,Unknown,01DVP6ABA6MSWJCVWVWF647HS6,accessory,"[chloésmall, marcie, leather, saddle, bag]"
65873,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,Unknown,Unknown,Unknown,01DVP6ABA6YSCYQP2SY055EW1J,accessory,"[chloésmall, marcie, leather, saddle, bag]"
65874,01DPGXRAZBHQYMCX47N55TP8PF,Unknown,Unknown,Unknown,Unknown,01E1FCZQH97DYHV4VPYV8F05AG,bottom,"[j, crewtall, asymmetrical, pencil, skirt, eve..."
65875,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,Unknown,Unknown,Unknown,01E2GX7WKRDKH05FS3NR4C25C4,top,"[reformationeddy, top]"
65876,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,Unknown,Unknown,Unknown,01E2KHK2TK75HF0APE38K8DH4R,top,"[reformationeddy, top]"


In [78]:
products_with_outfit_3 = products_with_outfit_2.copy()

Transform list to string for name_information, product_information, brand+name

In [79]:
def list_to_str(df, col):
    for i in range(len(df)):
        if df[col][i] == 'Unknown':
            continue
        lst = [word for word in df[col][i] if word != 'Unknown']
        final_str = " ".join(lst)
        df[col][i] = final_str

In [80]:
list_to_str(products_with_outfit_3, 'brand_category')
list_to_str(products_with_outfit_3, 'name')
list_to_str(products_with_outfit_3, 'product_information')
list_to_str(products_with_outfit_3, 'brand+name')

In [81]:
products_with_outfit_3.head()

,product_id,brand_category,name,product_information,outfit_type,outfit_id,outfit_item_type,brand+name
0,01EX0PN4J9WRNZH5F93YEX6QAF,unknown,khadi stripe shirt signature shirt,unknown two khadi stripe shirt signature shirt...,top,Unknown,Unknown,unknown
1,01F0C4SKZV6YXS3265JMC39NXW,unknown,ruffle market dress loopy pink sistine tomato,unknown collina strada ruffle market dress loo...,onepiece,Unknown,Unknown,unknown
2,01EY4Y1BW8VZW51BWG5VZY82XW,unknown,ibi slip raw red knit sneaker woman,unknown cariuma ibi slip raw red knit sneaker ...,shoe,Unknown,Unknown,unknown
3,01EY50E27A0P5V6KCW01XPDB43,unknown,ibi slip black knit sneaker woman,unknown cariuma ibi slip black knit sneaker wo...,shoe,Unknown,Unknown,unknown
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,unknown,catiba pro skate black suede canvas contrast t...,unknown cariuma catiba pro skate black suede c...,shoe,Unknown,Unknown,unknown


In [82]:
products_with_outfit_3.tail()

,product_id,brand_category,name,product_information,outfit_type,outfit_id,outfit_item_type,brand+name
65872,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,Unknown,Unknown,Unknown,01DVP6ABA6MSWJCVWVWF647HS6,accessory,chloésmall marcie leather saddle bag
65873,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,Unknown,Unknown,Unknown,01DVP6ABA6YSCYQP2SY055EW1J,accessory,chloésmall marcie leather saddle bag
65874,01DPGXRAZBHQYMCX47N55TP8PF,Unknown,Unknown,Unknown,Unknown,01E1FCZQH97DYHV4VPYV8F05AG,bottom,j crewtall asymmetrical pencil skirt everyday ...
65875,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,Unknown,Unknown,Unknown,01E2GX7WKRDKH05FS3NR4C25C4,top,reformationeddy top
65876,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,Unknown,Unknown,Unknown,01E2KHK2TK75HF0APE38K8DH4R,top,reformationeddy top


In [83]:
products_with_outfit_3['final_text'] = products_with_outfit_3['product_information'] + " " + \
                                       products_with_outfit_3['brand+name']

In [84]:
products_with_outfit_3 = products_with_outfit_3[['product_id', 'outfit_type', 'outfit_id', 
                                                 'outfit_item_type', 'final_text']]

In [85]:
products_with_outfit_3.head()

,product_id,outfit_type,outfit_id,outfit_item_type,final_text
0,01EX0PN4J9WRNZH5F93YEX6QAF,top,Unknown,Unknown,unknown two khadi stripe shirt signature shirt...
1,01F0C4SKZV6YXS3265JMC39NXW,onepiece,Unknown,Unknown,unknown collina strada ruffle market dress loo...
2,01EY4Y1BW8VZW51BWG5VZY82XW,shoe,Unknown,Unknown,unknown cariuma ibi slip raw red knit sneaker ...
3,01EY50E27A0P5V6KCW01XPDB43,shoe,Unknown,Unknown,unknown cariuma ibi slip black knit sneaker wo...
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,shoe,Unknown,Unknown,unknown cariuma catiba pro skate black suede c...


In [86]:
products_with_outfit_3.tail()

,product_id,outfit_type,outfit_id,outfit_item_type,final_text
65872,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,01DVP6ABA6MSWJCVWVWF647HS6,accessory,Unknown chloésmall marcie leather saddle bag
65873,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,01DVP6ABA6YSCYQP2SY055EW1J,accessory,Unknown chloésmall marcie leather saddle bag
65874,01DPGXRAZBHQYMCX47N55TP8PF,Unknown,01E1FCZQH97DYHV4VPYV8F05AG,bottom,Unknown j crewtall asymmetrical pencil skirt e...
65875,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,01E2GX7WKRDKH05FS3NR4C25C4,top,Unknown reformationeddy top
65876,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,01E2KHK2TK75HF0APE38K8DH4R,top,Unknown reformationeddy top


In [87]:
products_with_outfit_4 = products_with_outfit_3.copy()

In [88]:
products_with_outfit_4['final_text'] = products_with_outfit_4['final_text'].str.split(r'[\s\:\/]+')

In [89]:
for i in range(len(products_with_outfit_4)):
    lst = [word for word in products_with_outfit_4['final_text'][i] if word != 'unknown' and word != 'Unknown']
    final_str = " ".join(lst)
    products_with_outfit_4['final_text'][i] = final_str

In [90]:
products_with_outfit_4.tail()

,product_id,outfit_type,outfit_id,outfit_item_type,final_text
65872,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,01DVP6ABA6MSWJCVWVWF647HS6,accessory,chloésmall marcie leather saddle bag
65873,01DT0DKMM6G7HDJS12QCWK5X4H,Unknown,01DVP6ABA6YSCYQP2SY055EW1J,accessory,chloésmall marcie leather saddle bag
65874,01DPGXRAZBHQYMCX47N55TP8PF,Unknown,01E1FCZQH97DYHV4VPYV8F05AG,bottom,j crewtall asymmetrical pencil skirt everyday ...
65875,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,01E2GX7WKRDKH05FS3NR4C25C4,top,reformationeddy top
65876,01DPKMJFK6ESTB74DTB2HSB1S1,Unknown,01E2KHK2TK75HF0APE38K8DH4R,top,reformationeddy top


#### Correct the Outfit Category Using Existing Data in Outfit Dataframe

In [91]:
num_unmatch = set()
num_with_outfit = set()

for i in range(len(products_with_outfit_4)):
    if products_with_outfit_4.loc[i, 'outfit_item_type'] != 'Unknown' and products_with_outfit_4.loc[i, 'outfit_type'] != 'Unknown':
        num_with_outfit.add(products_with_outfit_4.loc[i, 'product_id'])
        if products_with_outfit_4.loc[i, 'outfit_type'] != products_with_outfit_4.loc[i, 'outfit_item_type']:
            num_unmatch.add(products_with_outfit_4.loc[i, 'product_id'])
            
print(len(num_unmatch))
print(len(num_with_outfit))

93
769


In [92]:
print(f'The accuracy rate of the matching is {round((1 - len(num_unmatch)/len(num_with_outfit)), 2)*100}%.')

The accuracy rate of the matching is 88.0%.


In [93]:
for i in range(len(products_with_outfit_4)):
    if products_with_outfit_4.loc[i, 'outfit_item_type'] != 'Unknown':
        if products_with_outfit_4.loc[i, 'outfit_type'] != products_with_outfit_4.loc[i, 'outfit_item_type']:
            products_with_outfit_4.loc[i, 'outfit_type'] = products_with_outfit_4.loc[i, 'outfit_item_type']

In [94]:
products_with_outfit_5 = products_with_outfit_4.copy()

#### Drop Duplicated and Irrelevant Records 

In [95]:
products_with_outfit_5.duplicated().sum()

53

In [96]:
len(products_with_outfit_5)

65877

In [97]:
products_with_outfit_5 = products_with_outfit_5.drop_duplicates()

In [98]:
len(products_with_outfit_5)

65824

In [99]:
len(products_with_outfit_5[products_with_outfit_5['outfit_type'] == 'Unknown'])

5427

In [100]:
len(products_with_outfit_5[products_with_outfit_5['outfit_type'] == 'others'])

733

In [101]:
products_with_outfit_6 = products_with_outfit_5[products_with_outfit_5['outfit_type'] != 'Unknown' ]
products_with_outfit_6.reset_index(inplace=True)

In [102]:
len(products_with_outfit_6)

60397

In [103]:
products_with_outfit_6 = products_with_outfit_6[products_with_outfit_6['outfit_type'] != 'others' ]
products_with_outfit_6.reset_index(inplace=True)

In [104]:
len(products_with_outfit_6)

59664

Finally, we had 59664 records for the recommendation system. 

In [105]:
products_with_outfit_6.head()

,level_0,index,product_id,outfit_type,outfit_id,outfit_item_type,final_text
0,0,0,01EX0PN4J9WRNZH5F93YEX6QAF,top,Unknown,Unknown,two khadi stripe shirt signature shirt signatu...
1,1,1,01F0C4SKZV6YXS3265JMC39NXW,onepiece,Unknown,Unknown,collina strada ruffle market dress loopy pink ...
2,2,2,01EY4Y1BW8VZW51BWG5VZY82XW,shoe,Unknown,Unknown,cariuma ibi slip raw red knit sneaker woman ib...
3,3,3,01EY50E27A0P5V6KCW01XPDB43,shoe,Unknown,Unknown,cariuma ibi slip black knit sneaker woman ibi ...
4,4,4,01EY6DWHC2W5HPNEGXKEJ4A1CX,shoe,Unknown,Unknown,cariuma catiba pro skate black suede canvas co...


In [106]:
products_with_outfit_6 = products_with_outfit_6.drop(['index', 'level_0'], axis=1)

In [107]:
products_with_outfit_6.head()

,product_id,outfit_type,outfit_id,outfit_item_type,final_text
0,01EX0PN4J9WRNZH5F93YEX6QAF,top,Unknown,Unknown,two khadi stripe shirt signature shirt signatu...
1,01F0C4SKZV6YXS3265JMC39NXW,onepiece,Unknown,Unknown,collina strada ruffle market dress loopy pink ...
2,01EY4Y1BW8VZW51BWG5VZY82XW,shoe,Unknown,Unknown,cariuma ibi slip raw red knit sneaker woman ib...
3,01EY50E27A0P5V6KCW01XPDB43,shoe,Unknown,Unknown,cariuma ibi slip black knit sneaker woman ibi ...
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,shoe,Unknown,Unknown,cariuma catiba pro skate black suede canvas co...


In [108]:
products_with_outfit_6.to_csv('cleaned_df.csv')

### Use CountVectorizer Method to Generate Vector

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

doc = list(products_with_outfit_6['final_text'].values)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(doc)
vec_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), 
                      index=products_with_outfit_6.product_id)

In [110]:
vec_df

,00p,00pm,00s,01cts,01k,01sc,02ct,02in,02k,03cts,...,ûªs,ÿan,œwarrior,œæ,šâ,šã,žâ,δione,ﬂat,ﬂowy
product_id,,,,,,,,,,,,,,,,,,,,,
01EX0PN4J9WRNZH5F93YEX6QAF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01F0C4SKZV6YXS3265JMC39NXW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01EY4Y1BW8VZW51BWG5VZY82XW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01EY50E27A0P5V6KCW01XPDB43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01EY6DWHC2W5HPNEGXKEJ4A1CX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01DT0DKMM6G7HDJS12QCWK5X4H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01DT0DKMM6G7HDJS12QCWK5X4H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01DPGXRAZBHQYMCX47N55TP8PF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


There were over 30,000 columns.

### Use Singular Value Decomposition

For Singular Value Decomposition, we used a python package TruncatedSVD in the sklearn library.

See more at https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/

In [111]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=3000)
# We reduced the dimension to about 10% of the original dimension
reduced_svd_features = svd.fit_transform(X)
reduced_df = pd.DataFrame(reduced_svd_features, index=products_with_outfit_6.product_id)
reduced_df = reduced_df.reset_index()

In [112]:
reduced_df_with_outfit = pd.merge(products_with_outfit_6, reduced_df, on='product_id')

In [113]:
reduced_df_with_outfit_2 = reduced_df_with_outfit.copy()

### Calculate the Cosine Similarity and Make Recommendation

We calculated the cosine similarity between two vectors using `scipy.spatial.distance.cosine`

See more at: https://www.kite.com/python/answers/how-to-find-the-cosine-similarity-between-two-vectors-in-python, and https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html

In [114]:
top_product = reduced_df_with_outfit_2[reduced_df_with_outfit_2['outfit_type'] == 'top']
top_product = top_product.reset_index()

bottom_product = reduced_df_with_outfit_2[reduced_df_with_outfit_2['outfit_type'] == 'bottom']
bottom_product = bottom_product.reset_index()

accessory_product = reduced_df_with_outfit_2[reduced_df_with_outfit_2['outfit_type'] == 'accessory']
accessory_product = accessory_product.reset_index()

shoe_product = reduced_df_with_outfit_2[reduced_df_with_outfit_2['outfit_type'] == 'shoe']
shoe_product = shoe_product.reset_index()

onepiece_product = reduced_df_with_outfit_2[reduced_df_with_outfit_2['outfit_type'] == 'onepiece']
onepiece_product = onepiece_product.reset_index()

In [115]:
top_product.head()

,index,product_id,outfit_type,outfit_id,outfit_item_type,final_text,0,1,2,3,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,01EX0PN4J9WRNZH5F93YEX6QAF,top,Unknown,Unknown,two khadi stripe shirt signature shirt signatu...,2.472271,-0.204293,0.128468,-0.558937,...,0.000793,-0.002254,0.039469,-0.004395,-0.039943,-0.036600,0.028593,-0.033227,0.032901,-0.011503
1,8,01EWTKPJNHRRDJZ9BVJWJV1T6Q,top,Unknown,Unknown,maia bergman lubi something nice jean begin lo...,1.094109,0.148905,-0.468256,0.207040,...,0.001565,0.018337,0.014979,-0.010883,0.002826,0.010305,0.001315,-0.020399,0.018886,-0.026354
2,11,01EWTKP0QZADY85ZMFZAH4Q122,top,Unknown,Unknown,maia bergman claudia onyx everybody favourite ...,1.338435,0.125712,-0.493426,0.130652,...,-0.025739,0.048480,0.007637,-0.049888,-0.016930,0.010469,-0.011461,0.032781,-0.016445,0.020622
3,12,01EWTKP97SE8DTWEYY5F19HSVE,top,Unknown,Unknown,maia bergman shani shani come pair top feature...,1.788386,0.036594,-0.512344,-0.127276,...,0.015658,-0.038762,-0.046567,0.033589,0.001010,-0.044635,-0.000355,0.039774,-0.048176,-0.055309
4,18,01EVF493Z4BS2QSGBQ27ECRJ2N,top,Unknown,Unknown,misa noa top cut micro pleat chiffon loose fit...,3.063136,0.153008,-1.366268,0.085338,...,-0.004682,-0.000889,-0.009231,-0.011416,0.010347,-0.017036,0.009041,-0.015547,0.011364,-0.011103


In [116]:
def preprocess_query(user_query):
    user_query = user_query.strip()
    user_query = user_query.lower()
    user_query = remove_stopwords(user_query)
    user_query = lemmatize_sentence(user_query)
    return user_query

The logic of the `search` function is:
1. Find out the most matched product in each category with the highest cosine similarity using the vector of input query and existing products
2. Check if the mosted matched product has an outfit id. If yes, then return the product id in each category with the same outfit id.
3. Compare the cosine similarity of onepiece, top, and bottom to determine whether the outfit contains onepiece only or contains top and bottom only
4. Return the product id in each category

In [117]:
def search(user_query: str):
    """
    user_query is a string that is passed in by the user, 
    and this function returns a dictionary of outfit results. Example:
    search("pleated casual skirt") -> { "top": "...",
    "bottom": "...",
    "shoe": "..."
    } 
    """
    user_query = preprocess_query(user_query)
    query_vec_df = vectorizer.transform([user_query])
    query_svd = svd.transform(query_vec_df)
    
    return_dict = {}
    
    # onepiece
    cos_op = 0
    op_product_id = None
    op_outfit_id = None
    op_idx = 0
    for i in range(len(onepiece_product)):
        cos_similarity = 1 - scipy.spatial.distance.cosine(query_svd, onepiece_product.iloc[i, 6:])
        if cos_similarity > cos_op:
            cos_op = cos_similarity
            op_product_id = onepiece_product.loc[i, 'product_id']
            op_outfit_id = onepiece_product.loc[i, 'outfit_id']
            op_idx = i
    
    if op_outfit_id != 'Unknown':
        matched_df = outfit_2[outfit_2['outfit_id'] == op_outfit_id]
        matched_df = matched_df.reset_index()
        for i in range(len(matched_df)):
            cate = matched_df.loc[i, 'outfit_item_type']
            pid = matched_df.loc[i, 'product_id']
            return_dict[cate] = pid
        return return_dict
    
    # top
    cos_top = 0
    top_product_id = None
    top_outfit_id = None
    top_idx = 0
    for i in range(len(top_product)):
        cos_similarity = 1 - scipy.spatial.distance.cosine(query_svd, top_product.iloc[i, 6:])
        if cos_similarity > cos_top:
            cos_top = cos_similarity
            top_product_id = top_product.loc[i, 'product_id']
            top_outfit_id = top_product.loc[i, 'outfit_id']
            top_idx = i
            
    if op_outfit_id != 'Unknown':
        matched_df = outfit_2[outfit_2['outfit_id'] == top_outfit_id]
        matched_df = matched_df.reset_index()
        for i in range(len(matched_df)):
            cate = matched_df.loc[i, 'outfit_item_type']
            pid = matched_df.loc[i, 'product_id']
            return_dict[cate] = pid
        return return_dict
    
    # bottom
    cos_bot = 0
    bot_product_id = None
    bot_outfit_id = None
    bot_idx = 0
    for i in range(len(bottom_product)):
        cos_similarity = 1 - scipy.spatial.distance.cosine(query_svd, bottom_product.iloc[i, 6:])
        if cos_similarity > cos_bot:
            cos_bot = cos_similarity
            bot_product_id = bottom_product.loc[i, 'product_id']
            bot_outfit_id = bottom_product.loc[i, 'outfit_id']
            bot_idx = i
            
    if bot_outfit_id != 'Unknown':
        matched_df = outfit_2[outfit_2['outfit_id'] == bot_outfit_id]
        matched_df = matched_df.reset_index()
        for i in range(len(matched_df)):
            cate = matched_df.loc[i, 'outfit_item_type']
            pid = matched_df.loc[i, 'product_id']
            return_dict[cate] = pid
        return return_dict
        
    # shoe
    cos_shoe = 0
    shoe_product_id = None
    shoe_outfit_id = None
    shoe_idx = 0
    for i in range(len(shoe_product)):
        cos_similarity = 1 - scipy.spatial.distance.cosine(query_svd, shoe_product.iloc[i, 6:])
        if cos_similarity > cos_shoe:
            cos_shoe = cos_similarity
            shoe_product_id = shoe_product.loc[i, 'product_id']
            shoe_outfit_id = shoe_product.loc[i, 'outfit_id']
            shoe_idx = i
            
    if shoe_outfit_id != 'Unknown':
        matched_df = outfit_2[outfit_2['outfit_id'] == shoe_outfit_id]
        matched_df = matched_df.reset_index()
        for i in range(len(matched_df)):
            cate = matched_df.loc[i, 'outfit_item_type']
            pid = matched_df.loc[i, 'product_id']
            return_dict[cate] = pid
        return return_dict
    
    # accessory
    cos_accessory = 0
    accessory_product_id = None
    accessory_outfit_id = None
    accessory_idx = 0
    for i in range(len(accessory_product)):
        cos_similarity = 1 - scipy.spatial.distance.cosine(query_svd, accessory_product.iloc[i, 6:])
        if cos_similarity > cos_accessory:
            cos_accessory = cos_similarity
            accessory_product_id = accessory_product.loc[i, 'product_id']
            accessory_outfit_id = accessory_product.loc[i, 'outfit_id']
            accessory_idx = i
            
    if accessory_outfit_id != 'Unknown':
        matched_df = outfit_2[outfit_2['outfit_id'] == accessory_outfit_id]
        matched_df = matched_df.reset_index()
        for i in range(len(matched_df)):
            cate = matched_df.loc[i, 'outfit_item_type']
            pid = matched_df.loc[i, 'product_id']
            return_dict[cate] = pid
        return return_dict
    
    # determine whether onepiece or tom + bottom
    if cos_op > cos_top and cos_op > cos_bot:
        return_dict['onepiece'] = op_product_id
    else:
        return_dict['top'] = top_product_id
        return_dict['bottom'] = bot_product_id
        
    return_dict['shoe'] = shoe_product_id
    return_dict['accessory'] = accessory_product_id
    
    return return_dict

In [118]:
def show_results(return_dict):
    final_dict = {}
    for cate, pid in return_dict.items():
        outfit_df = outfit_2[outfit_2['product_id'] == pid]
        if not outfit_df.empty:
            final_dict[cate] = outfit_df.iloc[0, :]['product_full_name']
        else:
            product_name = list(products_1[products_1['product_id'] == pid]['name'].values)
            if product_name:
                final_dict[cate] = product_name[0]
            else:
                final_dict[cate] = pid
    return final_dict

### Examples for Testing

Example 1

In [119]:
result_dict = search('slim fitting, straight leg pant with a center back zipper and slightly cropped leg')

In [120]:
result_dict

{'top': '01DS3NA7ECEY16CA144VM14Y4X',
 'bottom': '01EDYCNSC1837PDVKT5D8PSX15',
 'shoe': '01EF2G6W4D04YVBDNCGFTZFD3Z',
 'accessory': '01EHB0CPX1GEBP8QASDWNX27RV'}

In [121]:
final_dict = show_results(result_dict)

In [122]:
final_dict

{'top': 'Open-Back Cropped Vinyl Vest',
 'bottom': 'Kerrigan Pant',
 'shoe': 'Flat Westside Pant',
 'accessory': 'The Straight with Clean Pocket'}

Example 2

In [123]:
result_dict_2 = search('pleated casual skirt')
final_dict_2 = show_results(result_dict_2)
print(result_dict_2)
print(final_dict_2)

{'top': '01EDYAKKQJNSQJ2S02B44VS1PQ', 'bottom': '01EQ9NX4QHM9D9A9GQN5F97T9R', 'shoe': '01EZ7QC3A5TKKANTTSCNTTHZXE', 'accessory': '01EJ78Q46PQDH60P2AX2Q6E5XW'}
{'top': 'Cami Skirt', 'bottom': 'KNIFE PLEAT SKIRT', 'shoe': 'Mahi', 'accessory': 'Lea Skirt New Arrival'}


Example 3

In [124]:
result_dict_3 = search('turtleneck sweater designed in an oversized silhouette')
final_dict_3 = show_results(result_dict_3)
print(result_dict_3)
print(final_dict_3)

{'top': '01E2KZQJ9H2HCADQGF6V8YK7J4', 'bottom': '01EA34T8MDQPHH17PXZ95BDKBE', 'shoe': '01EZ7G8FVD8WZJRDZW73QZ3G9Y', 'accessory': '01EMJSMJHQ9YWYCE50TVCHMM1P'}
{'top': 'Chenille Turtleneck High/Low Sweater', 'bottom': 'Oversized Crop Tee', 'shoe': 'Alysis', 'accessory': 'DUMONT SWEATER'}
